In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
 # QUERY ( smaller)
 import numpy
 vet1 = numpy.load('/content/gdrive/My Drive/copy_detection/3files30oct.npz', allow_pickle= True)
 total_images_hash_values1 = vet1['a']
 total_images_features1 = vet1['b']

In [ ]:
total_images_hash_values1[1][1]

array(['00001110', '00001110', '00011000', '00111000', '00001111',
       '00000110', '00001000', '01111000', '00001111', '10011111',
       '00001000', '01110000', '00000010', '00000011', '00000000',
       '11100000'], dtype='<U8')

In [ ]:
 # Database (bigger)

 vet2 = numpy.load('/content/gdrive/My Drive/copy_detection/Shubham_28oct/157.npz', allow_pickle= True)
 total_images_hash_values2 = vet2['a']
 total_images_features2 = vet2['b']

In [ ]:
total_images_hash_values2[1][1]

['00000000',
 '00100010',
 '00000110',
 '00000000',
 '10000011',
 '11000001',
 '11100111',
 '10000000',
 '10000011',
 '11000001',
 '11100011',
 '10000000',
 '10000011',
 '11000011',
 '11000001',
 '11100000']

In [ ]:
import numpy as np

In [ ]:
#stacking all images all keypoints in to a single vector
img_feat1 = total_images_features1[0]
for i in total_images_features1[1:]:
    img_feat1 = np.vstack((img_feat1,i))
print(img_feat1.shape)

(4500, 256)


In [ ]:
#stacking all images all keypoints in to a single vector
img_feat2 = total_images_features2[0]
for i in total_images_features2[1:]:
    img_feat2 = np.vstack((img_feat2,i))
print(img_feat2.shape)

(227429, 256)


In [ ]:
import scipy
import random

random.seed(101)

from scipy.cluster.vq import kmeans,vq #vq stands for vector quantization
k= 100 #maximum clusters can be 50
#voc1,variance=kmeans(img_feat1,k)        #voc decides no of clusters to be formed, based on distace
                                     # 50 is no of iteration

In [ ]:
# saving
#import numpy
#numpy.savez('/content/gdrive/My Drive/Pixy_project/full_157_information.npz', a=total_images_hash_values1, b=total_images_features1, c = voc1, d = im_features, e = database_words)

In [ ]:

import numpy
vet = numpy.load('/content/gdrive/My Drive/copy_detection/Shubham_28oct/final_db_with_centroids.npz', allow_pickle= True)
#total_images_hash_values1 = vet['a']
#total_images_features1 = vet['b']
voc1 = vet['c']
#im_features1 = vet['d']
#database_words1 = vet['e']

In [ ]:
import os

In [ ]:
#clustering all the keypoints of all images
database_words1 = []
im_features1=np.zeros((len(total_images_features1),k),"float32")
for i in range(0,len(total_images_features1)):
    words1,distance=vq(total_images_features1[i],voc1) #vector quantization
    database_words1.append(words1)
    for w in words1:
        im_features1[i][w]+=1

In [ ]:
#clustering all the keypoints of all images
database_words2 = []
im_features2=np.zeros((len(total_images_features2),k),"float32")
for i in range(0,len(total_images_features2)):
    words1,distance=vq(total_images_features2[i],voc1) #vector quantization
    database_words2.append(words1)
    for w in words1:
        im_features2[i][w]+=1

In [ ]:
files_path="/content/gdrive/My Drive/copy_detection/Pixy_project/original_image"
val =sorted([int(i.split(".")[0]) for i in os.listdir(files_path)]) #os.listdir wont give you file order
images1 = [str(i)+".jpg" for i in val] #getting files in order

In [ ]:
files_path="/content/gdrive/My Drive/copy_detection/Pixy_project/copydays_original"
val =sorted([int(i.split(".")[0]) for i in os.listdir(files_path)]) #os.listdir wont give you file order
images2 = [str(i)+".jpg" for i in val] #getting files in order

In [ ]:
print(len(images1))
print(len(images2))

3
157


In [ ]:
images1=images1[0:len(total_images_features1)]
images2=images2[0:len(total_images_features2)]

In [ ]:
print(len(images1))
print(len(total_images_features1))

print(len(images2))
print(len(total_images_features2))

3
3
157
157


In [ ]:
image_vocab1 = {} #filepath,vocab
for file_name,im_feature in zip(images1,im_features1):
    image_vocab1[file_name] = list(im_features1)

In [ ]:
image_vocab2 = {} #filepath,vocab
for file_name,im_feature in zip(images2,im_features2):
    image_vocab2[file_name] = list(im_features2)

In [ ]:
def to_list(im_features1):
  apple=[]
  for i in range(0,len(im_features1)):
    apple.append(list(im_features1[i]))
  return apple

In [ ]:
#im_features1 =[list(i) for i in im_features1]

In [ ]:
im_features1=to_list(im_features1)
database_words1=to_list(database_words1)
im_features2=to_list(im_features2)
database_words2=to_list(database_words2)

In [ ]:
# INVERTED INDEX

In [ ]:
# GERENRATE WEIGHTS

import math
dict1={}
for i in range(0,len(im_features2)):
  apple=im_features2[i]
  for index,value in enumerate(apple):
    if value > 0:
      if index in dict1:
        dict1[index]=dict1[index] +1
      else:
        dict1[index]=1
for key,value in dict1.items():
  IDF= math.log(len(database_words2) / value)
  dict1[key]=IDF

idf_dict=dict1

In [ ]:
from collections import defaultdict
inverted_indx1 = defaultdict(list) 
for image_id,feature,kp in zip(list(range(len(images1))),im_features1,database_words1): #101.jpg, im_feature,430kp
    for indx,count in enumerate(feature): #for every feature in im_features
        if int(count)>0: #if fequency value is >0
            # no = int(image_id.split(".")[0]) # image no
            hashes = [{ind:list(total_images_hash_values1[image_id][ind])}  for ind,j in enumerate(kp) if j == indx]
            #hashes= [list(i) for i in hashes]
            image_no=images1[image_id]
            image_no=image_no.split('.')
            imgae_no=image_no[0]
            inverted_indx1[indx].append([imgae_no,hashes])

In [ ]:
dict1={}
images={}
for image_id,feature,kp in zip(list(range(len(images1))),im_features1,database_words1):
    for indx,count in enumerate(feature): 
        if int(count)>0: 
            hashes = [list(total_images_hash_values1[image_id][ind])  for ind,j in enumerate(kp) if j == indx]
            image_no=images1[image_id]
            image_no=image_no.split('.')
            imgae_no=image_no[0]
            images[imgae_no]=hashes
            idf_weight=idf_dict[indx]
            dict1[indx]={'images':images,'idf_weight':idf_weight}

In [ ]:
import numpy
numpy.savez('/content/gdrive/My Drive/shubham.npz', a=dict1)

In [ ]:
 import numpy
 vet1 = numpy.load('/content/gdrive/My Drive/shubham.npz', allow_pickle= True)
 idf_dic = vet1['a']


In [ ]:
# saving
import json
with open('/content/gdrive/My Drive/shubham.json', 'w') as fp:
    json.dump(dict1, fp)

# Loading
with open('/content/gdrive/My Drive/shubham.json', 'r') as fp:
  data = json.load(fp) 

In [ ]:
data

In [ ]:
d = idf_dic.flatten()[0]

In [ ]:
d

{0: {'idf_weight': 0.47127832667773606,
  'images': {'200800': [['00001100',
     '00111110',
     '00011110',
     '00001000',
     '00001100',
     '00111110',
     '00011111',
     '00001000',
     '00000010',
     '00000000',
     '00100111',
     '00000000',
     '10000011',
     '11000011',
     '11000011',
     '10000000'],
    ['00001100',
     '00111110',
     '00011110',
     '00001000',
     '00001100',
     '00111110',
     '00011111',
     '00001000',
     '00000010',
     '00000000',
     '00100111',
     '00000000',
     '10000011',
     '11000011',
     '11000011',
     '10000000'],
    ['00001100',
     '00111110',
     '00011110',
     '00001000',
     '00001100',
     '00111110',
     '00011111',
     '00001000',
     '00000010',
     '00000000',
     '00100111',
     '00000000',
     '10000011',
     '11000011',
     '11000011',
     '10000000'],
    ['00001100',
     '00111110',
     '00011110',
     '00001000',
     '00001100',
     '00111110',
     '00011111',
  

In [ ]:
new=d[0]
new

{0: {'idf_weight': 0.47127832667773606,
  'images': {'200800': [['00001100',
     '00111110',
     '00011110',
     '00001000',
     '00001100',
     '00111110',
     '00011111',
     '00001000',
     '00000010',
     '00000000',
     '00100111',
     '00000000',
     '10000011',
     '11000011',
     '11000011',
     '10000000'],
    ['00001100',
     '00111110',
     '00011110',
     '00001000',
     '00001100',
     '00111110',
     '00011111',
     '00001000',
     '00000010',
     '00000000',
     '00100111',
     '00000000',
     '10000011',
     '11000011',
     '11000011',
     '10000000'],
    ['00001100',
     '00111110',
     '00011110',
     '00001000',
     '00001100',
     '00111110',
     '00011111',
     '00001000',
     '00000010',
     '00000000',
     '00100111',
     '00000000',
     '10000011',
     '11000011',
     '11000011',
     '10000000'],
    ['00001100',
     '00111110',
     '00011110',
     '00001000',
     '00001100',
     '00111110',
     '00011111',
  

In [ ]:
dict1[0]

{'idf_weight': 0.47127832667773606,
 'images': {'200800': [['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111',
    '00000000',
    '10000011',
    '11000011',
    '11000011',
    '10000000'],
   ['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111',
    '00000000',
    '10000011',
    '11000011',
    '11000011',
    '10000000'],
   ['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111',
    '00000000',
    '10000011',
    '11000011',
    '11000011',
    '10000000'],
   ['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111

In [ ]:
dict1.keys()

dict_keys([2, 14, 16, 18, 24, 27, 28, 31, 33, 46, 48, 65, 69, 74, 76, 0, 1, 3, 4, 6, 9, 10, 11, 12, 17, 20, 23, 25, 26, 30, 34, 35, 36, 37, 39, 41, 42, 43, 44, 49, 50, 52, 54, 57, 58, 59, 61, 66, 67, 68, 75, 77, 80, 81, 7, 13, 45, 56, 62, 71, 79, 82])

In [ ]:
dict(dict1[0])

{'idf_weight': 0.47127832667773606,
 'images': {'200800': [['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111',
    '00000000',
    '10000011',
    '11000011',
    '11000011',
    '10000000'],
   ['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111',
    '00000000',
    '10000011',
    '11000011',
    '11000011',
    '10000000'],
   ['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111',
    '00000000',
    '10000011',
    '11000011',
    '11000011',
    '10000000'],
   ['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111

In [ ]:
dict1[0]

{'idf_weight': 0.47127832667773606,
 'images': {'200800': [['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111',
    '00000000',
    '10000011',
    '11000011',
    '11000011',
    '10000000'],
   ['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111',
    '00000000',
    '10000011',
    '11000011',
    '11000011',
    '10000000'],
   ['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111',
    '00000000',
    '10000011',
    '11000011',
    '11000011',
    '10000000'],
   ['00001100',
    '00111110',
    '00011110',
    '00001000',
    '00001100',
    '00111110',
    '00011111',
    '00001000',
    '00000010',
    '00000000',
    '00100111

In [ ]:
dict1.items()

dict_items([(2, {'images': {'200800': [['00001100', '00111110', '00011110', '00001000', '00001100', '00111110', '00011111', '00001000', '00000010', '00000000', '00100111', '00000000', '10000011', '11000011', '11000011', '10000000'], ['00001100', '00111110', '00011110', '00001000', '00001100', '00111110', '00011111', '00001000', '00000010', '00000000', '00100111', '00000000', '10000011', '11000011', '11000011', '10000000'], ['00001100', '00111110', '00011110', '00001000', '00001100', '00111110', '00011111', '00001000', '00000010', '00000000', '00100111', '00000000', '10000011', '11000011', '11000011', '10000000'], ['00001100', '00111110', '00011110', '00001000', '00001100', '00111110', '00011111', '00001000', '00000010', '00000000', '00100111', '00000000', '10000011', '11000011', '11000011', '10000000']], '200900': [['00001110', '00001110', '00011000', '00111000', '00001111', '00000110', '00001000', '01111000', '00001111', '10011111', '00001000', '01110000', '00000010', '00000011', '000

In [ ]:
for i,j in dict1.items():
  for n,m in j.items():
    try:
      for s,t in m.items():
        print(s)
    except:
      pass

215500
215500
215500
200800
200900
200800
215500
200800
215500
215500
215500
215500
215500
215500
215500
215500
200900
215500
215500
200900
200900
200900
215500
215500
215500
200900
215500
215500
215500
215500
200900
200900
215500
200900
200900
200900
200900
215500
200900
200900
215500
200900
200900
215500
200900
215500
200900
200900
200900
200900
215500
215500
215500
200900
215500
215500
215500
215500
215500
215500
215500
215500


In [ ]:
from collections import defaultdict
inverted_indx2 = defaultdict(list) 
for image_id,feature,kp in zip(list(range(len(images2))),im_features2,database_words2): #101.jpg, im_feature,430kp
    for indx,count in enumerate(feature): #for every feature in im_features
        if int(count)>0: #if fequency value is >0
            # no = int(image_id.split(".")[0]) # image no
            hashes = [{ind:total_images_hash_values2[image_id][ind]}  for ind,j in enumerate(kp) if j == indx]
            image_no=images2[image_id]
            image_no=image_no.split('.')
            imgae_no=image_no[0]
            inverted_indx2[indx].append([imgae_no,hashes])

In [ ]:
# save to new variables
inverted_indx=inverted_indx1
q_inverted_indx=inverted_indx2

In [ ]:
#IDF Logic for 1 cluster
#IDF_0 = math.log(total images/ no of images present with cluster zero)

In [ ]:
# GERENRATE WEIGHTS

import math
dict1={}
for i in range(0,len(im_features2)):
  apple=im_features2[i]
  for index,value in enumerate(apple):
    if value > 0:
      if index in dict1:
        dict1[index]=dict1[index] +1
      else:
        dict1[index]=1
for key,value in dict1.items():
  IDF= math.log(len(database_words2) / value)
  dict1[key]=IDF

idf_dict=dict1

In [ ]:
ls3=[]
for i in list(inverted_indx.keys()):
  for j in list(q_inverted_indx.keys()):
    if i==j: # if same cluster
      dict2={}
      for k in range(0,len(inverted_indx[i])): # example :length inverted_indx[1] == 25  
        a_image=inverted_indx[i][k][0] #save image no. 
        a_keypoints=inverted_indx[i][k][1] #save all the keypoints for above a_image
        A=[]
        for l in a_keypoints:
          A.append(str(list(l.values())[0])) # only hash value in list ( its in array format now)
        dict1={}

NameError: ignored

In [ ]:
# COMPARE ALL IMAGES OF DB to ALL IMAGES OF QUERY

In [ ]:
#q_inverted_indx  # cluster # image # keypoint # hash
#inverted_indx
ls3=[]
for i in list(inverted_indx.keys()):
  for j in list(q_inverted_indx.keys()):
    if i==j: # if same cluster
      dict2={}
      for k in range(0,len(inverted_indx[i])): # example :length inverted_indx[1] == 25  
        a_image=inverted_indx[i][k][0] #save image no. 
        a_keypoints=inverted_indx[i][k][1] #save all the keypoints for above a_image
        A=[]
        for l in a_keypoints:
          A.append(str(list(l.values())[0])) # only hash value in list ( its in array format now)
        dict1={}
        for n in range(0,len(q_inverted_indx[j])): # similarly for query, but under database loop ( so that 1 query image gets checked with vs all data base images)
          b_image=q_inverted_indx[j][n][0]
          b_keypoints=q_inverted_indx[j][n][1]
          B=[]
          for m in b_keypoints:
            B.append(str(list(m.values())[0]))
          counter=0 
          for t1 in B: # if query hash present in db hash just add 1
            if t1 in A:
              counter=counter+1
          s=idf_dict[i] # load weights
          # example: ith cluster 1st value ([3.43, 0])
          dict1[b_image] = counter*s
        dict2[a_image] = dict1
      ls3.append(dict2)        

In [ ]:
ls3

[{'0': {'200000': 13.018368191398588,
   '200100': 0.0,
   '200300': 0.0,
   '200400': 0.0,
   '200500': 0.0,
   '200600': 0.0,
   '200700': 0.0,
   '201000': 0.0,
   '201300': 0.0,
   '201400': 0.0,
   '202300': 0.0,
   '202600': 0.0,
   '203100': 0.0,
   '203300': 0.0,
   '203400': 0.0,
   '204500': 0.0,
   '205000': 0.0,
   '205400': 0.0,
   '205500': 0.0,
   '205600': 0.0,
   '205700': 0.0,
   '205800': 0.0,
   '206400': 0.0,
   '206600': 0.0,
   '206700': 0.0,
   '206800': 0.0,
   '207500': 0.0,
   '207600': 0.0,
   '207700': 0.0,
   '207800': 0.0,
   '207900': 0.0,
   '208000': 0.0,
   '208200': 0.0,
   '208400': 0.0,
   '208800': 0.0,
   '208900': 0.0,
   '209000': 0.0,
   '209200': 0.0,
   '209300': 0.0,
   '209600': 0.0,
   '209700': 0.0,
   '209800': 0.0,
   '209900': 0.0,
   '210000': 0.0,
   '210100': 0.0,
   '210200': 0.0,
   '210300': 0.0,
   '210600': 0.0,
   '210700': 0.0,
   '210800': 0.0,
   '210900': 0.0,
   '211100': 0.0,
   '211300': 0.0,
   '211600': 0.0,
   '2117

In [ ]:
# summing all results ( all of clusters results to 1)

In [ ]:
it = 1
final_score = {}
#getting the unique file names across all clusters.
for data_base_image in ls3:
  #first iteration.
  if it == 1:
    unique_file_names = set(data_base_image.keys())
    it+=1
  else:
    buffer_set = set(data_base_image.keys())
    #getting uncommen elements.
    un_commen_elements = buffer_set - unique_file_names
    #adding uncommen elements to the unique list.
    for element in un_commen_elements:
      unique_file_names.add(element)
    it+=1
it = 1
#unique file_name
for data_base_image_name in unique_file_names:
  #elements in the cluster
  for cluster in ls3:
    if data_base_image_name in cluster.keys():
      # in a cluster we are looking for quary image
      for quary_file_name in cluster[data_base_image_name].keys():
        sum = 0
        #searching combination on different clusters
        for cluster_iteration in ls3:
          if data_base_image_name in cluster_iteration.keys():
            if quary_file_name in cluster_iteration[data_base_image_name].keys():
              sum+=cluster_iteration[data_base_image_name][quary_file_name]
              final_score[str(data_base_image_name)+"-"+str(quary_file_name)] = sum
            else:
              pass

In [ ]:
# TAKE ONLY BEST MATCH

In [ ]:
def take_dic(dic):
    import pandas as pd
    lis_left = []
    lis_right = []
    score = []
    data=pd.DataFrame.from_dict(dic, orient ='index')
    data.reset_index(level=0, inplace=True)
    data['left'] = data["index"].str.split("-", n = 1, expand = True)[0]
    data['right'] = data["index"].str.split("-", n = 1, expand = True)[1]
    data['left']=data['left'].astype('category',copy=False)
    for value in data['left'].unique():
        lis_left.append(value)
        df=data[data['left']==value]
        lis_right.append(df.loc[df[0].idxmax()]['right'])
        score.append(df.loc[df[0].idxmax()][0])
    return pd.DataFrame.from_dict({"left":lis_left,"right":lis_right,"score":score})
Best_match=take_dic(final_score)


In [ ]:
# check results

In [ ]:
def metric(Best_match):
  True_positive_count=0
  False_positive_count=0
  for i in range(0,len(Best_match)):
    if int(Best_match.iloc[i]['left']==Best_match.iloc[i]['right']): #or int(Best_match.iloc[i]['left']==Best_match.iloc[i]['right']):
      True_positive_count += 1
    else:
      False_positive_count += 1
  print('True_positive_count=',True_positive_count)
  print('False_positive_count=',False_positive_count)
  print('Accuracy=',(True_positive_count/len(Best_match))*100)

metric(Best_match)

True_positive_count= 0
False_positive_count= 11
Accuracy= 0.0


In [ ]:
Best_match

,left,right,score
0,6,200000,0.000000
1,3,200000,0.000000
2,7,200000,0.000000
3,4,200000,0.000000
4,10,200000,0.000000
5,8,200000,0.000000
6,0,200000,43.178734
7,5,200000,0.000000
8,9,200000,0.000000
9,1,200000,0.000000
